In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline                                           
from sklearn.preprocessing import StandardScaler                                     
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold as SKF
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.metrics import roc_auc_score as AUC
from sklearn.base import clone
from sklearn import tree
from sklearn.model_selection import ParameterGrid, GridSearchCV
from joblib import Parallel, delayed
from multiprocessing import Pool
from tqdm import tqdm
from scipy.spatial import distance
from skbio.stats.ordination import pcoa
import umap
from sklearn.metrics import make_scorer
from sklearn import linear_model
import warnings
warnings.filterwarnings("ignore")

# Get data

In [6]:
# read meta data
df_meta = pd.read_csv('../../our_data/meta_data.csv', index_col=0)
df_meta = df_meta[df_meta.Diet=='Inulin']
df_meta = df_meta[df_meta.Day != 0] # remove day 0

# read SCFA data
df_scfa = pd.read_csv('../../our_data/SCFA.csv', index_col=0)

# read bacterial abundance (species level)
df_bac = pd.read_csv('../../our_data/16S_absolute_abundance_species.csv', index_col=0)

# find common samples
common_samples = list(set(df_meta.index).intersection(df_scfa.index).intersection(df_bac.index))
df_meta = df_meta.loc[common_samples]
df_scfa = df_scfa.loc[common_samples]
df_bac = df_bac.loc[common_samples]

# remove species that are constant across all samples
df_bac = df_bac[list(df_bac.std()[df_bac.std()>0].index)]
df_bac = df_bac[['Bacteroides-acidifaciens','Muribaculaceae','Faecalibaculum','Parasutterella','Bacteroides']]

# Run ElasticNet

## intrapolation

In [7]:
results = []
for k,group_to_exclude in enumerate(['A','B','C','D']):

    # split train/test data
    mice_to_keep = list(set(df_meta[df_meta.RandomizedGroup!=group_to_exclude].MiceID))
    samples_to_keep = list(set(df_meta[df_meta.MiceID.isin(mice_to_keep)].index))
    mice_to_exclude = list(set(df_meta[df_meta.RandomizedGroup==group_to_exclude].MiceID))
    samples_to_exclude = list(set(df_meta[df_meta.MiceID.isin(mice_to_exclude)].index))

    # get weights of training sets
    xdata_train = np.asarray(df_bac.loc[samples_to_keep].values)
    xdata_test = np.asarray(df_bac.loc[samples_to_exclude].values)
    
    # run random forest regression with weights
    for scfa in ['Acetate','Propionate','Butyrate']:                
        ydata_train = np.asarray(df_scfa.loc[samples_to_keep, scfa])
        ydata_test = np.asarray(df_scfa.loc[samples_to_exclude, scfa])
        param_grid = {"elasticnet__alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "elasticnet__l1_ratio": [.01, .1, .3, .5, .7, .9, .99]}
        
        clf = ElasticNet(tol=1e-4,random_state=0,max_iter=1000000,normalize=False)
        pipe = make_pipeline(StandardScaler(), clf) 
        CV = GridSearchCV(pipe, param_grid, scoring='r2', cv=5, n_jobs=-1, verbose=2)
        CV.fit(xdata_train, ydata_train)

        print('Intrapolation, group %s, %s, best score and parameter combination = '%(group_to_exclude, scfa))
        print(CV.best_score_)    
        print(CV.best_params_)    
        print('\n')

        # predict training set
        ydata_train_predicted = CV.predict(xdata_train)
        ydata_test_predicted = CV.predict(xdata_test)

        for sample_, obs_, pred_ in zip(samples_to_keep, ydata_train, ydata_train_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['intrapolation', scfa, group_to_exclude, 'train', sample_, day_, obs_, pred_])
        for sample_, obs_, pred_ in zip(samples_to_exclude, ydata_test, ydata_test_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['intrapolation', scfa, group_to_exclude, 'test', sample_, day_, obs_, pred_])

Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 222 out of 245 | elapsed:    1.9s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    1.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s


Intrapolation, group A, Acetate, best score and parameter combination = 
-0.09608736235857404
{'elasticnet__alpha': 10, 'elasticnet__l1_ratio': 0.01}


Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s


Intrapolation, group A, Propionate, best score and parameter combination = 
0.13571195014421153
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.01}


Fitting 5 folds for each of 49 candidates, totalling 245 fits


...

[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s


Intrapolation, group D, Acetate, best score and parameter combination = 
0.040129147646034126
{'elasticnet__alpha': 1, 'elasticnet__l1_ratio': 0.99}


Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s


Intrapolation, group D, Propionate, best score and parameter combination = 
0.07831875608158057
{'elasticnet__alpha': 1, 'elasticnet__l1_ratio': 0.01}


Fitting 5 folds for each of 49 candidates, totalling 245 fits
Intrapolation, group D, Butyrate, best score and parameter combination = 
-0.046558981106130326
{'elasticnet__alpha': 1, 'elasticnet__l1_ratio': 0.1}




[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    0.2s finished


## extrapolation

In [8]:
for k,vendor_to_exclude in enumerate(['Beijing','Guangdong','Hunan','Shanghai']):
        
    # split train/test data
    mice_to_keep = list(set(df_meta[df_meta.Vendor!=vendor_to_exclude].MiceID))
    samples_to_keep = list(set(df_meta[df_meta.MiceID.isin(mice_to_keep)].index))
    mice_to_exclude = list(set(df_meta[df_meta.Vendor==vendor_to_exclude].MiceID))
    samples_to_exclude = list(set(df_meta[df_meta.MiceID.isin(mice_to_exclude)].index))

    # get weights of training sets
    xdata_train = np.asarray(df_bac.loc[samples_to_keep].values)
    xdata_test = np.asarray(df_bac.loc[samples_to_exclude].values)
    
    # run random forest regression with weights
    for scfa in ['Acetate','Propionate','Butyrate']:                
        ydata_train = np.asarray(df_scfa.loc[samples_to_keep, scfa])
        ydata_test = np.asarray(df_scfa.loc[samples_to_exclude, scfa])
        param_grid = {"elasticnet__alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "elasticnet__l1_ratio": [.01, .1, .3, .5, .7, .9, .99]}
        
        clf = ElasticNet(tol=1e-4,random_state=0,max_iter=1000000,normalize=False)
        pipe = make_pipeline(StandardScaler(), clf) 
        CV = GridSearchCV(pipe, param_grid, scoring='r2', cv=5, n_jobs=-1, verbose=2)
        CV.fit(xdata_train, ydata_train)

        print('Extrapolation, vendor %s, %s, best score and parameter combination = '%(vendor_to_exclude, scfa))
        print(CV.best_score_)    
        print(CV.best_params_)    
        print('\n')   

        # predict training set
        ydata_train_predicted = CV.predict(xdata_train)
        ydata_test_predicted = CV.predict(xdata_test)

        for sample_, obs_, pred_ in zip(samples_to_keep, ydata_train, ydata_train_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['extrapolation', scfa, vendor_to_exclude, 'train', sample_, day_, obs_, pred_])
        for sample_, obs_, pred_ in zip(samples_to_exclude, ydata_test, ydata_test_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['extrapolation', scfa, vendor_to_exclude, 'test', sample_, day_, obs_, pred_])


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s


Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s


Extrapolation, vendor Beijing, Acetate, best score and parameter combination = 
0.02713356494362864
{'elasticnet__alpha': 1, 'elasticnet__l1_ratio': 0.3}


Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s


...

[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s


Extrapolation, vendor Shanghai, Acetate, best score and parameter combination = 
-0.23903764754964535
{'elasticnet__alpha': 10, 'elasticnet__l1_ratio': 0.3}


Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s


Extrapolation, vendor Shanghai, Propionate, best score and parameter combination = 
-0.048266239198727814
{'elasticnet__alpha': 10, 'elasticnet__l1_ratio': 0.3}


Fitting 5 folds for each of 49 candidates, totalling 245 fits
Extrapolation, vendor Shanghai, Butyrate, best score and parameter combination = 
-0.1026147354163609
{'elasticnet__alpha': 1, 'elasticnet__l1_ratio': 0.7}




[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    0.3s finished


## save results to file

In [9]:
df_prediction = pd.DataFrame(results, columns=['PerturbationType','SCFA','Permutation','PredictionType','SampleID','Day','ObservedValue','PredictedValue'])
df_prediction.to_csv('rf_prediction_predictor_speciesonly_elasticnet_degraders.csv')